In [ ]:
## Non-parametric calculation of the significance of changes in intensity for intensity Hovmoller plots
# based in large part on china_rain_diff.ipynb, and previous library of bootstrap codes (bootstrap.py)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import netCDF4 as nc
import matplotlib.pyplot as plt
%matplotlib notebook
%load_ext autotime
from bootstrap import *
from shutil import copyfile

In [2]:
#load hovmoller storage of intensity
ferretpath = "/Users/Siwen/Desktop/ferret/bin/"
hovfile = "RDA_int_hov.nc"
file1 = ferretpath + hovfile
hovf = nc.Dataset(file1, 'r')

#load hovmoller climatology (mean, standard deviation and standard deviation of sample mean)
climofile = "RDA_int_climo.nc"
file2 = ferretpath + climofile
climof = nc.Dataset(file2, 'r')

time: 16.8 ms


In [10]:
## BOOTSTRAPPING ##
#from previous testing, determined that the autocorrelation time scale of rainfall is between
#2 and 3 days. Will test both and see effect on p-values.

#General expectation would be that longer block lengths will lead to fewer significant p-values
#(smaller effective sample size

##REWRITTEN SEPTEMBER 26, 2016
#previous version had to run for potentially days at a time, whereas now it saves each new row as it completes it.

#daysmth is defined above
niter = 2000 #how many iterations each time?

p1_beg = [1951,1980]
p1_end = [1979,1993]
p2_beg = [1980,1994]
p2_end = [2007,2007]

dayrange = 15 #what range of days are we smoothing over?
latrange = 5 #what range of latitude are we smoothing over? (1 degree increments)

filepath = "RDA_int_diff_bootstrap.nc"
backuppath = "/Users/Siwen/Desktop/Ferret/bin/"
fullname = backuppath + filepath

if os.path.isfile(fullname):
    os.remove(fullname)

fileout = nc.Dataset(fullname, "w")

## PRIMARY FRONTS ONLY ##
#actual bootstrapping loop - loads data from periods of interest, then performs bootstrapping calculations.
for p1_b, p1_e, p2_b, p2_e in zip(p1_beg, p1_end, p2_beg, p2_end):
    
    years_p1= str(p1_b-1900) + str("{0:0=2d}".format((p1_e-1900)%100))
    print(years_p1)
    years_p2= str(p2_b-1900) + str("{0:0=2d}".format((p2_e-1900)%100))
    print(years_p2)
    
    int_p1 = hovf['int_1_hov_' + years_p1][:]
    int_p2 = hovf['int_1_hov_' + years_p2][:]
    
    pval = np.empty([int_p1.shape[0], int_p1.shape[0]])
    
    #BOOTSTRAPPING CALCULATION OF P-VALUE OF INTENSITY CHANGES BETWEEN TIME PERIODS
    for i in range(200,int_p1.shape[0]):
        
        for j in range(int_p1.shape[1]):
            
            s1 = collect(int_p1, i,j, 15, 5)
            s2 = collect(int_p2, i,j, 15, 5)
            
            print(s1)
            print(s2)
            
            if ((len(s1) > 0) & (len(s2) > 0)):
                #pval[i,j] = 
                qq = bs_means_diff(s1, s2, niter)
                print(qq)
                time.sleep(3)
                
            else:
                pval[i,j] = np.NaN
    
    ## SAVE OUTPUT AS NETCDF FILE
    
## ALL FRONTS ##
        
hovf.close()
climof.close()
fileout.close()

5179
8007
[-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 33.37701482358186 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --]
[-- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- 44.34933196581327 -- -- -- -- -- -- --
 -- -- 50.58632293343544 -- -- -- -- -- -- 65.9067059357961 -- -- -- -- --
 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
 -- -- --]
[-- -- -- -- -- -- -- -- -- 34.19167822599411 37.50148799571585 -- -- -- --
 -- -- -- -- -- --

KeyboardInterrupt: 

time: 20.9 s
